In [1]:
import sys
sys.path.append('..')
from services import reddit
from dotenv import load_dotenv

load_dotenv()


d:\Projects\fastflow\fastflowenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
reddit_service = reddit.RedditService()

borikto


In [3]:
df = await reddit_service.search_reddit("Best skills to look for in ML Engineers", post_limit=100)

In [4]:
df = df[df['body'].astype(bool)]
json_documents = df.to_dict('records')
df

,title,score,id,subreddit,url,num_comments,body,created
1,Linguist look for ML engineers/datascientists ...,1,1d76m1g,MLjobs,https://www.reddit.com/r/MLjobs/comments/1d76m...,8,"Hello, I'm a linguist, I look for meaningful p...",2024-06-03 20:33:31
2,What are the paramount skills/tools for a data...,5,1dphmko,mlops,https://www.reddit.com/r/mlops/comments/1dphmk...,5,Have an extensive amount of data engineering a...,2024-06-27 09:00:03
3,[D] Best community/website to find ML engineer...,32,1cook89,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,48,I've been searching for a machine learning eng...,2024-05-10 18:08:16
4,Is it common for data engineers to have profic...,25,1b2qxvx,dataengineering,https://www.reddit.com/r/dataengineering/comme...,18,I have been scrolling through the job postings...,2024-02-29 09:29:40
5,Engineering student Proficient in ML looking f...,3,1cyo5o4,StartUpIndia,https://www.reddit.com/r/StartUpIndia/comments...,5,I'm an engineering student from India.\nI have...,2024-05-23 14:19:21
...,...,...,...,...,...,...,...,...
95,Are you looking for an ongoing help in program...,0,1djdmf8,careerchange,https://www.reddit.com/r/careerchange/comments...,0,Hey there - I've been working in tech for over...,2024-06-19 13:21:18
96,[B2B] A skilled and ambitious engineer looking...,3,1awu4z5,mspjobs,https://www.reddit.com/r/mspjobs/comments/1awu...,2,Hi!\n\nI'm a 28-year-old engineer from Europe ...,2024-02-22 07:16:22
97,Looking for the best laptop between $1000-$150...,3,1e7iap6,SuggestALaptop,https://www.reddit.com/r/SuggestALaptop/commen...,10,The recommendations set out by my university a...,2024-07-20 05:29:18
98,Best search engine to look for a WI cabin,0,1bgov7r,wisconsin,https://www.reddit.com/r/wisconsin/comments/1b...,4,"As the title says, what’s the best search engi...",2024-03-17 09:45:57


In [77]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core import Document
from llama_index.core.node_parser import JSONNodeParser
from typing import List, Dict, Any
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
import os 
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor

from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.response_synthesizers import BaseSynthesizer

GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

llm = Gemini(model="models/gemini-1.5-flash", api_key=GEMINI_API_KEY)
embed_model = GeminiEmbedding(model_name="models/embedding-001", 
                              api_key=GEMINI_API_KEY,
                              title="this is a document")
# Settings.llm = llm
# Settings.embed_model = embed_model
documents = [Document(text=json.dumps(doc)) for doc in json_documents]
nodeparser = JSONNodeParser()
nodes = nodeparser.get_nodes_from_documents(documents)
index = VectorStoreIndex(nodes=nodes, embed_model=embed_model)


ValueError: Model models/aqa does not support content generation, only ['generateAnswer'].

In [ ]:

from llama_index.core import PromptTemplate

qa_prompt = PromptTemplate(
    "Here are some documents containing information, they could be a JSON-like paragraphs or normal paragraphs, \n"
    "interpret them as needed.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Answer the following query based on the information, do not use any prior knowledge.\n"
    "Query: {query_str}\n"
    "Answer: "
)


class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: Gemini
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)

        context_str = "__\n\n__".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return dict(response = str(response), nodes = nodes)

In [78]:


retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=20,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

query_engine = RAGStringQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)


In [79]:
query1 = "What are most discussed topic?"
response = query_engine.custom_query(query1)

print("Query 1:", query1)
print(response['response'])

Query 1: What are most discussed topic?
The most discussed topics are:

* **Data Engineering:**  There are multiple posts asking about data engineering courses, skills needed for data engineers, and whether ML proficiency is necessary for data engineers.
* **AI/ML:**  Many posts are about AI/ML engineering jobs, skills needed for AI/ML engineers, and how to learn AI/ML.
* **Software Engineering:**  There are posts about niche software engineering skills in Europe and how to transition from a general full-stack developer to a more specialized role.
* **Career Advice:**  Several posts are about career advice, including finding the best country to move to in the EU, finding a partner for a data-driven project, and transitioning to a new career path. 
* **Game Development:**  One post discusses the best game engine to learn for non-programmers. 



In [75]:
for node in response['nodes']:
    print(node.text)

title What are the best subreddits for women looking to improve their social skills?
score 2
id 10gdkn3
subreddit SubSimGPT2Interactive
url https://www.reddit.com/r/SubSimGPT2Interactive/comments/10gdkn3/what_are_the_best_subreddits_for_women_looking_to/
num_comments 313
body [removed]
created 2023-01-20 02:48:42
title What niche software engineering skill is currently/becoming highly looked for in Europe?
score 50
id 19a11vk
subreddit cscareerquestionsEU
url https://www.reddit.com/r/cscareerquestionsEU/comments/19a11vk/what_niche_software_engineering_skill_is/
num_comments 49
body Hello,  


I am a software engineer currently 2 YoE and I was thinking of making the move to Europe in 1/2 years time. I am just a general full-stack web developer with a bit of knowledge in cloud and ML, however I was wondering if it would be smarter to increase my chances of employment by becoming more specialised in an area. I find the whole space of software interesting so going down a single path is fin

In [67]:
df.sort_values(by='num_comments', ascending=False)[['url', 'num_comments']].head(10).values

array([['https://www.reddit.com/r/leagueoflegends/comments/k7yl34/after_months_of_work_and_no_sleep_its_finally/',
        344],
       ['https://www.reddit.com/r/SubSimGPT2Interactive/comments/10gdkn3/what_are_the_best_subreddits_for_women_looking_to/',
        313],
       ['https://www.reddit.com/r/cscareerquestionsEU/comments/19b9f3y/looking_for_best_country_to_move_in_eu/',
        266],
       ['https://www.reddit.com/r/ycombinator/comments/1deynl5/feel_useless_without_coding_skills_as_ml/',
        126],
       ['https://www.reddit.com/r/godot/comments/16lqnqq/im_looking_for_the_easiest_game_engine_to_learn/',
        113],
       ['https://www.reddit.com/r/womenintech/comments/1b4bqay/finally_ready_to_leave_tech_at_age_55/',
        110],
       ['https://www.reddit.com/r/Piracy/comments/ovwrwg/new_torrent_search_engine_in_town_bitsearchto/',
        107],
       ['https://www.reddit.com/r/lostarkgame/comments/w0d3dc/what_to_look_for_in_bus_drivers_minimum/',
        98],
     

In [14]:
import services
import services.mem_rag
from importlib import reload
reload(services.mem_rag)
from services.mem_rag import InMemoryRAG

rag = InMemoryRAG()


In [15]:
rag.add_documents(df)

In [3]:
from services.youtube_scraper import fetch_youtube_comments

# Test the fetch_youtube_comments function
video_id = "SAiyopkF22M"  # Replace with a valid YouTube video ID
max_results = 50  # Adjust as needed

comments = fetch_youtube_comments(video_id, max_results)

print(f"Fetched {len(comments)} comments:")
for i, comment in enumerate(comments[:5], 1):  # Print first 5 comments as a sample
    print(f"\nComment {i}:")
    print(f"Author: {comment['author']}")
    print(f"Text: {comment['text']}")
    print(f"Likes: {comment['likes']}")
    print(f"Published at: {comment['published_at']}")

print(f"\n... and {len(comments) - 5} more comments.")


Fetched 50 comments:

Comment 1:
Author: @CookwithMunu
Text: God save this song from badsha  🙂
Likes: 0
Published at: 2024-08-03T13:34:19Z

Comment 2:
Author: @Rerurusglamdust
Text: i hate this songs
Likes: 0
Published at: 2024-08-02T19:17:53Z

Comment 3:
Author: @Rerurusglamdust
Text: faltu songs
Likes: 0
Published at: 2024-08-02T19:17:42Z

Comment 4:
Author: @MdLatifurIslamLabib
Text: Who listening this song in 2024 😅
Likes: 0
Published at: 2024-08-01T05:09:16Z

Comment 5:
Author: @unmeshakhajbage4165
Text: "Kyu" and "haye"of sonu nigam has my heart 
And that "haa" of shreya ghosal is just awesome❤❤
Likes: 0
Published at: 2024-07-29T13:53:07Z

... and 45 more comments.


In [2]:
# Import the function from the tweets service
from services.tweets import get_tweet_replies

# Example tweet ID
tweet_id = '1816862466816496101'

# Fetch replies for the tweet
replies = get_tweet_replies(tweet_id)

if replies is not None:
    print(f"Found {len(replies)} replies to tweet {tweet_id}")
    for reply in replies:
        print(f"Reply from @{reply.user.screen_name}: {reply.full_text}")
else:
    print("Failed to fetch replies")


Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

In [15]:

import os
from dotenv import load_dotenv
load_dotenv()
    # Set up authentication using environment variables
import requests
import os
import json
import tweepy

auth = tweepy.OAuth2AppHandler(
    os.environ.get("TWITTER_API_KEY"),
    os.environ.get("TWITTER_API_SECRET_KEY"),
)
api = tweepy.API(auth)

tweet_id = 1816892137377587684

In [19]:
from playwright.async_api import async_playwright


def scrape_tweet(url: str) -> dict:
    """
    Scrape a single tweet page for Tweet thread e.g.:
    https://twitter.com/Scrapfly_dev/status/1667013143904567296
    Return parent tweet, reply tweets and recommended tweets
    """
    _xhr_calls = []

    def intercept_response(response):
        """capture all background requests and save them"""
        # we can extract details from background requests
        if response.request.resource_type == "xhr":
            _xhr_calls.append(response)
        return response

    with async_playwright() as pw:
        browser = pw.chromium.launch(headless=False)
        context = browser.new_context(viewport={"width": 1920, "height": 1080})
        page = context.new_page()

        # enable background request intercepting:
        page.on("response", intercept_response)
        # go to url and wait for the page to load
        page.goto(url)
        page.wait_for_selector("[data-testid='tweet']")

        # find all tweet background requests:
        tweet_calls = [f for f in _xhr_calls if "TweetResultByRestId" in f.url]
        for xhr in tweet_calls:
            data = xhr.json()
            return data['data']['tweetResult']['result']



print(scrape_tweet("https://twitter.com/Scrapfly_dev/status/1664267318053179398"))

AttributeError: __enter__